# Test 1: Basic Text Extraction with Layout Parser

**Goal:** Verify Google Document AI Layout Parser works and extracts raw text from PDFs

**What this test does:**
- Connects to Google Document AI with Layout Parser processor
- Processes a PDF document
- Extracts text content page by page
- Saves results to JSON file

**Layout Parser Benefits:**
- Better structure detection than basic OCR
- Preserves document layout hierarchy
- Detects paragraphs, headers, and text blocks
- Ideal for academic textbooks and complex documents


## Step 1: Install Dependencies

Run this cell to install required packages for Google Colab.


In [ ]:
%pip install -q google-cloud-documentai python-dotenv openai anthropic pdf2image Pillow
print("✅ All dependencies installed!")


: 

## Step 2: Upload Credentials

Upload your Google Cloud service account JSON file.


In [ ]:
from google.colab import files
import json
import os

print("📤 Please upload your Google Cloud credentials JSON file...")
uploaded = files.upload()

# Save credentials
creds_filename = list(uploaded.keys())[0]
credentials_content = json.loads(uploaded[creds_filename].decode('utf-8'))

# Write to temporary file
with open('docai-credentials.json', 'w') as f:
    json.dump(credentials_content, f)

os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = 'docai-credentials.json'
print(f"✅ Credentials saved: {creds_filename}")


## Step 3: Configure Environment

Set your Google Cloud project ID and Layout Parser processor ID.


In [ ]:
# Configuration - UPDATE THESE VALUES
DOCAI_PROJECT_ID = "your-project-id-here"  # Replace with your project ID
DOCAI_PROCESSOR_ID = "your-processor-id-here"  # Replace with your Layout Parser processor ID
DOCAI_LOCATION = "us"  # us, eu, or asia

# Set environment variables
os.environ['DOCAI_PROJECT_ID'] = DOCAI_PROJECT_ID
os.environ['DOCAI_PROCESSOR_ID'] = DOCAI_PROCESSOR_ID
os.environ['DOCAI_LOCATION'] = DOCAI_LOCATION

print(f"✅ Configuration set:")
print(f"   Project: {DOCAI_PROJECT_ID}")
print(f"   Processor: {DOCAI_PROCESSOR_ID}")
print(f"   Location: {DOCAI_LOCATION}")


## Step 4: Clone Repository and Load Utils

Clone the repository to access utility functions.


In [ ]:
# Clone repository
!git clone https://github.com/abhii-01/docai-extraction-test.git
%cd docai-extraction-test

# Import utilities
import sys
from pathlib import Path
sys.path.append(str(Path.cwd()))

from utils.docai_client import get_client_from_env

print("✅ Repository cloned and utilities loaded")


## Step 5: Verify Setup

Test connection to Google Document AI.


In [ ]:
print("🔍 Verifying Document AI setup...\n")

try:
    client = get_client_from_env()
    client.verify_setup()
    print("\n✅ Setup verified! Ready to process documents.")
except Exception as e:
    print(f"\n❌ Setup verification failed: {e}")
    print("\nPlease check:")
    print("  1. Credentials file is valid")
    print("  2. Project ID is correct")
    print("  3. Processor ID is correct")
    print("  4. Layout Parser processor exists and is enabled")


## Step 6: Upload PDF for Testing

Upload your PDF file to process.


In [ ]:
print("📤 Please upload your PDF file...")
uploaded_pdfs = files.upload()

# Get the first uploaded PDF
pdf_filename = list(uploaded_pdfs.keys())[0]
pdf_path = pdf_filename

print(f"✅ PDF uploaded: {pdf_filename}")
print(f"   Size: {len(uploaded_pdfs[pdf_filename]) / 1024:.1f} KB")


## Step 7: Process Document with Layout Parser

Extract text from the PDF using Layout Parser processor.


In [ ]:
print(f"{'='*60}")
print("TEST 1: BASIC TEXT EXTRACTION")
print(f"{'='*60}\n")

# Process document
print(f"📄 Processing PDF with Layout Parser: {pdf_path}")
document = client.process_document(pdf_path)

# Extract full text
full_text = document.text
print(f"✅ Document processed successfully!")
print(f"   Total pages: {len(document.pages)}")
print(f"   Total characters: {len(full_text):,}")


## Step 8: Extract Page-by-Page Text

Extract text from each page individually.


In [ ]:
print("📝 Extracting text by page...\n")

pages_data = []

for page_num, page in enumerate(document.pages, 1):
    page_text = ""
    
    # Get text from paragraphs
    for paragraph in page.paragraphs:
        if paragraph.layout.text_anchor:
            # Extract text segments
            text_segments = []
            for segment in paragraph.layout.text_anchor.text_segments:
                start = segment.start_index
                end = segment.end_index
                text_segments.append(document.text[start:end])
            
            page_text += " ".join(text_segments) + "\n\n"
    
    pages_data.append({
        "page_number": page_num,
        "text": page_text.strip(),
        "char_count": len(page_text)
    })
    
    print(f"  Page {page_num}: {len(page_text):,} characters")

print(f"\n✅ Extracted text from {len(pages_data)} pages")


## Step 9: Preview Extracted Text

Display a preview of the extracted text.


In [ ]:
print("📄 TEXT PREVIEW (first 800 characters):")
print("=" * 60)
print(full_text[:800])
if len(full_text) > 800:
    print("...")
print("=" * 60)


## Step 10: Save Results to JSON

Save extraction results to a JSON file.


In [ ]:
import json
from pathlib import Path

# Build results
results = {
    "pdf_file": Path(pdf_path).name,
    "total_pages": len(document.pages),
    "total_characters": len(full_text),
    "full_text": full_text,
    "pages": pages_data
}

# Save to JSON
output_path = "test1_raw_text.json"
with open(output_path, 'w', encoding='utf-8') as f:
    json.dump(results, f, indent=2, ensure_ascii=False)

print(f"💾 Results saved to: {output_path}")
print(f"\n📊 Summary:")
print(f"  Total pages: {len(document.pages)}")
print(f"  Total characters: {len(full_text):,}")
print(f"  Average chars/page: {len(full_text) // len(document.pages):,}")


## Step 11: Download Results

Download the JSON results file to your computer.


In [ ]:
from google.colab import files

print("📥 Downloading results...")
files.download(output_path)
print(f"✅ Test 1 complete! Results downloaded.")
